In [11]:
bytes(chr(255).encode('utf-8'))

b'\xc3\xbf'

【问题】bytes(chr(255).encode('utf-8'))  ；输出：b'\xc3\xbf' ；为什么是2个字节，255应该是1个字节吧？

 - 混淆了“数值 255”和“Unicode 字符 U+00FF”

chr(255) 返回的是 Unicode 码点 255 对应的字符，即：
U+00FF → 字符 'ÿ'（拉丁小写字母 y 上加两点）
这是一个 Unicode 字符，不是“字节 0xFF”。

In [15]:
chr(255)

'ÿ'

In [36]:
print(bytes([255, 253]))
print(bytes([25]) == bytes([25]))

b'\xff\xfd'
True


In [23]:
res = 'ab一'.encode('utf-8')
print(res)
b1 = res[1:3]
b2 = res[3:4]
print(b1+b2)
# 在 Python 中，对 bytes 对象进行遍历会得到 整数（0~255），
# 所以不能直接用 list(b'...') 得到字节片段。
print(list(res))
# 它拆分成单字节的列表，即每个元素是一个长度为 1 的 bytes 对象
print([bytes([c]) for c in res])


b'ab\xe4\xb8\x80'
b'b\xe4\xb8'
[97, 98, 228, 184, 128]
[b'a', b'b', b'\xe4', b'\xb8', b'\x80']


In [33]:
b'a'.decode('utf-8')

'a'

In [4]:
import regex as re 
res  = re.escape("<|end|>")
print(res, type(res))
res2 = re.escape("<|st|>")

PAT = re.compile(f"{res}|{res2}")
tstr = "aaa<|end|>bbb<|st|>ccc<|st2|>"
mts = re.split(PAT, tstr)
print(mts)

<\|end\|> <class 'str'>
['aaa', 'bbb', 'ccc<|st2|>']


In [9]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

iter = re.finditer(PAT, "abc, efd higk")
for res in iter:
    print(res.group())

abc
,
 efd
 higk


## Merge Part

In [35]:
a = [1]
ps = zip(a[:-1], a[1:])
for v in ps:
    print(v)


In [34]:
a = [1, 5, 3]
ps = zip(a[:-1], a[1:])
for v in ps:
    print(v)

# 列表是可变对象（mutable），
# Python 无法为其生成稳定的哈希值（hash），
# 所以不允许用作字典的 key。
# dc = {[1]: 3, [1]: 4}
dc = {(1, 2): [1, 2, 3, 4]}
print(dc)
del dc[(1, 2)][1]
print(dc)
ls = dc[(1, 2)]
del ls[1]
print(dc)

print(len(ls))

(1, 5)
(5, 3)
{(1, 2): [1, 2, 3, 4]}
{(1, 2): [1, 3, 4]}
{(1, 2): [1, 4]}
2


In [6]:
ls = ['b', 'a', 'f', 'c']
dc = {'a': 9, 'b': 10, 'c': 10, 'd': 10}

def inverse_ord(key):
    return -ord(key)

def choose_best_key(d, tie_breaker):
    """
    d: dict, key -> count
    tie_breaker: function(key) -> comparable value (越小/越大越优先，取决于你的逻辑)
    
    返回 count 最大的 key；若 count 相同，则按 tie_breaker(key) 的值选择（默认 max 会选最大的 tie_breaker 值）
    如果你希望 tie_breaker 越小越好（比如字典序靠前），可以取负值或用 min，但这里我们调整 key 函数。
    """
    return max(d.items(), key=lambda item: (item[1], tie_breaker(item[0])))

print(max(ls))
print(max(dc))
print(choose_best_key(dc, inverse_ord))

f
d
('b', 10)


In [31]:
pylist = [5, 7, 22, 13, 9]
del pylist[1]
print(pylist)
pylist.pop(1)
print(pylist)

[5, 22, 13, 9]
[5, 13, 9]
0 5
1 13
2 9



【BPE的实现】
 - 思路是直接的：找最多的对
 - 合并之后，原有的对需要更新。那么，如何找到需要更新的对？
 - 每一次merge，都用新的merge规则，重新扫描生成新的对？效率低下
 - 【无思路】看之前的仓库，找提示。


伪代码：
 - tok 

```
# toks key should be bytes?
# toks = {' abc': 2, 'efd': 10, ',': 3, ' abe', ' ab一': 20}
# toks = {b'ab\xe4\xb8\x80': 20}
# toks = { [b'a', b'b', b'\xe4', b'\x80']: 20 }
pairs
for tok, freq in toks:
    tokpairs = zip(tok[:-1], tok[1:])
    for pair in tokpairs:
        pairs[pair] = pairs.get(pair, 0) + 1

max_pair = pairs_maxkey(pairs)
merges.add(max_pair)
toks_apply_merge(toks, merge)
vocabs[max_pair[0] + max_pair[1]] = tokenID
vocabSize++

def toks_apply_merge(toks, merge):
    for tok, freq in toks:
        for i in range(len(tok)):
            if (tok[i], tok[i+1]) == merge:
                tok[i] = tok[i] + tok[i+1]
                del tok[i+1]

    
    
```

## 字典序

Python 对**元组**按字典序（lexicographical order） 比较，规则如下：
 - 从左到右逐个比较元素；
 - 第一个不相等的元素决定大小；
 - 如果前面所有元素都相等，则较短的元组更小。


从左到右逐个元素比较
'BA' > 'B' 因为 'B' == 'B'，但 'A' > ''（其实这里 "BA" > "B" 是因为 `"B" 是 "BA" 的前缀，较短的排在前面）
 - Python 原生支持元组的字典序比较

【问题2】  
    如何对字典里的 key: count , 找到 count最大在 key，有多个时根据一个函数根据key进行比较

【边界情况】

>>> max([("A", "B"), ("A", "C"), ("B", "ZZ"), ("BA", "A")])
('BA', 'A')

>>> max([('BAA','A'), ('BA','A')])
('BAA','A')

>>> max([('BAA','A'), ('BA','Z')])
('BAA','A')

>>> max([('BAA','A'), ('BAA','Z')])
('BAA','Z')

In [12]:
pairs = {("A", "B"): 1, ("A", "C"):10,  ("BA", "A"):10, ("B", "ZZ"):10, ('Z', 'A'): 9}

max_key = max(pairs.items(), key=lambda x: (x[1], x[0]))[0]
print(max_key)

pairs = {("A", "B"): 1, ("A", "C"):10,  ("BA", "A"):10, ("BA", "Z"):10, ('Z', 'A'): 9}
max_key = max(pairs.items(), key=lambda x: (x[1], x[0]))[0]
print(max_key)

('BA', 'A')
('BA', 'Z')


## BPE-Train的实现问题


1、merges中的全是重复的 (' ', 't') pair.
  - 【原因】 bpe_merge函数  while i > 0 , 应该是 >= 0, 导致跳过合并，后续持续出现同样的最大pairs

In [39]:


def bpe_merge(mergeTokensDc:dict, merge):
    for tok, mergeToks in mergeTokensDc.items():
        mlen = len(mergeToks)
        if mlen <= 1:
            continue

        i = mlen - 2
        # Note: 越界问题
        while i >= 0:
            print(mergeToks[i],  merge[0], mergeToks[i] == merge[0])
            if mergeToks[i] == merge[0] and mergeToks[i+1] == merge[1]:
                mergeToks[i] = mergeToks[i] + mergeToks[i+1]
                del mergeToks[i+1]
            i -= 1

merge = (b' ', b't')
mergeTokensDc = {' teching': [bytes([c]) for c in ' teching'.encode('utf-8')]}
bpe_merge(mergeTokensDc, merge)
print(mergeTokensDc)


b'n' b' ' False
b'i' b' ' False
b'h' b' ' False
b'c' b' ' False
b'e' b' ' False
b't' b' ' False
b' ' b' ' True
{' teching': [b' t', b'e', b'c', b'h', b'i', b'n', b'g']}



2、【问题2】


```
    vocabs = {}
    for i in range(257):
	    vocabs[i] = ...
	vocabSize = len(vocabs)
    assert(len(vocabs) == vocabSize)
    while vocabSize < vocab_size:
	    # 断点调试：len(vocabs) = 257；vocabs[257] = b'\x80'
	    # 调试器看见 vocabs可以访问 511个元素 ？为什么？
        vocabs[vocabSize] = max_pair[0] + max_pair[1]
        vocabSize += 1
        if len(vocabs) != vocabSize:
            pass # 为什么进入这里
```
 【问题】为什么我循环加入，长度变得不一样了？我从空的dict加入 key-value；key每次都是连续的，不可能存在重复key
【原因】：写出简单的bug， `vocabSize+i` 导致不是连续的key；想法完全错误

```
    for i in range(256):
        vocabs[vocabSize+i] = bytes([i])
        vocabSize += 1
```


【问题3】

In [57]:
def get_most_frequent_pair(
    pair_counter: dict[tuple[int, int], int],
) -> tuple[int, int]:
    max_freq = max(pair_counter.values())
    candidates = [pair for pair, freq in pair_counter.items() if freq == max_freq]
    res = max(candidates)

    return res

pair_counter = {
    (b'2', b'4'): 77,
    (b' .', b'..'): 77,
}

print(get_most_frequent_pair(pair_counter))

max_pair = max(pair_counter.items(), key=lambda x: (x[1], x[0]))[0]
print(max_pair)


most_common_pair: tuple[bytes, bytes] | None = None
most_common_pair_count = 0
for pair, count in pair_counter.items():
    if (
        not most_common_pair
        or count > most_common_pair_count
        or (count == most_common_pair_count and pair > most_common_pair)
    ):
        most_common_pair = pair
        most_common_pair_count = count
print(most_common_pair)

sorted(pair_counter, key=lambda x: x[1], reverse=True)

(b'2', b'4')
(b'2', b'4')
(b'2', b'4')


[(b'2', b'4'), (b' .', b'..')]

In [52]:
[int(a) for a in b' .']

reference_merges = (b' .', b' .')
[int(a) for a in reference_merges[0]]

[32, 46]